#Import library and dataset

In [49]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/fbhsaid/DAV6300/main/Global%20Baseline%20Data.csv')
df.head()

,Unnamed: 0,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
0,Olivia,NaN,5,NaN,4.5,3.0,NaN
1,ZJY,4.0,3,NaN,4.0,4.0,NaN
2,LHR,5.0,3,NaN,4.0,NaN,NaN
3,ZHC,4.0,3,3.5,4.5,3.5,2.5
4,ZYC,3.0,4,NaN,3.0,3.0,NaN


#Data Transformations

In [50]:
df.index = ['Olivia', 'ZJY', 'LHR', 'ZHC','ZYC','Tianen']
df.columns =['Col_1', 'Huawei', 'Xiaomi', 'Vivo', 'Apple', 'Samsung', 'Sony']
df1 = df[['Huawei', 'Xiaomi', 'Vivo', 'Apple', 'Samsung', 'Sony']]
df1

,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
Olivia,NaN,5,NaN,4.5,3.0,NaN
ZJY,4.0,3,NaN,4.0,4.0,NaN
LHR,5.0,3,NaN,4.0,NaN,NaN
ZHC,4.0,3,3.5,4.5,3.5,2.5
ZYC,3.0,4,NaN,3.0,3.0,NaN
Tianen,5.0,4,NaN,4.5,4.0,NaN


In [51]:
df1 = df1.fillna(0)
df1

,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
Olivia,0.0,5,0.0,4.5,3.0,0.0
ZJY,4.0,3,0.0,4.0,4.0,0.0
LHR,5.0,3,0.0,4.0,0.0,0.0
ZHC,4.0,3,3.5,4.5,3.5,2.5
ZYC,3.0,4,0.0,3.0,3.0,0.0
Tianen,5.0,4,0.0,4.5,4.0,0.0


In [52]:
df1 = df1.T
df1

,Olivia,ZJY,LHR,ZHC,ZYC,Tianen
Huawei,0.0,4.0,5.0,4.0,3.0,5.0
Xiaomi,5.0,3.0,3.0,3.0,4.0,4.0
Vivo,0.0,0.0,0.0,3.5,0.0,0.0
Apple,4.5,4.0,4.0,4.5,3.0,4.5
Samsung,3.0,4.0,0.0,3.5,3.0,4.0
Sony,0.0,0.0,0.0,2.5,0.0,0.0


# Import KNN from scikit learn, set metric to cosine similiarity

In [53]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df1.values)
distances, indices = knn.kneighbors(df1.values, n_neighbors=3)

#Check that we can apply to our dataset successfully

In [54]:
indices

array([[0, 3, 1],
       [1, 3, 4],
       [2, 5, 3],
       [3, 1, 4],
       [4, 3, 1],
       [2, 5, 3]])

In [55]:
distances

array([[0.00000000e+00, 1.11457491e-01, 1.87921506e-01],
       [0.00000000e+00, 2.65006689e-02, 9.42007034e-02],
       [0.00000000e+00, 0.00000000e+00, 5.53886563e-01],
       [3.33066907e-16, 2.65006689e-02, 9.21774535e-02],
       [0.00000000e+00, 9.21774535e-02, 9.42007034e-02],
       [0.00000000e+00, 0.00000000e+00, 5.53886563e-01]])

Define two functions, first is to get phones names based on their index value, the second is to get similiar phones to a particular phone to recommend to someone looking for alternatives to that phone. To do that, we first get the indices of the similiar phones, then we apply the first fuction we defined to get phone names before finally returning them.

In [57]:
#Define function to get phone names based on their index
def getName(arr):
    return [df1.index[index] for index in arr]

#Define function to get similiar phones by getting their indices and applying the getName function we just defined.
def getsimphoneNames (phone):
  index_for_phone = df1.index.tolist().index(phone)
  sim_phones = indices[index_for_phone].tolist()
  if sim_phones[0] == index_for_phone:
    print('similiar phones to '+phone+' is', getName(sim_phones[1:]))
  if sim_phones[0] != index_for_phone:
    print('similiar phones to '+phone+' is', getName(sim_phones[:-1]))

#Apply function to Huawei for similiar phones to Huawei
getsimphoneNames ('Huawei')
#Apply function to all other phones
getsimphoneNames ('Xiaomi')
getsimphoneNames ('Apple')
getsimphoneNames ('Samsung')
getsimphoneNames ('Vivo')
getsimphoneNames ('Sony')

similiar phones to Huawei is ['Apple', 'Xiaomi']
similiar phones to Xiaomi is ['Apple', 'Samsung']
similiar phones to Apple is ['Xiaomi', 'Samsung']
similiar phones to Samsung is ['Apple', 'Xiaomi']
similiar phones to Vivo is ['Sony', 'Apple']
similiar phones to Sony is ['Vivo', 'Sony']


##Using LightFM

In [58]:
#Install LightFM first
!pip install LightFM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
#We are going to look at users as rows this time so I'm transposing the matrix back
df2 = df1.T
df2

,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
Olivia,0.0,5.0,0.0,4.5,3.0,0.0
ZJY,4.0,3.0,0.0,4.0,4.0,0.0
LHR,5.0,3.0,0.0,4.0,0.0,0.0
ZHC,4.0,3.0,3.5,4.5,3.5,2.5
ZYC,3.0,4.0,0.0,3.0,3.0,0.0
Tianen,5.0,4.0,0.0,4.5,4.0,0.0


In [60]:
#Import libraries
from scipy.sparse import csr_matrix
from scipy import sparse
from lightfm import LightFM
#Fit lightFM model
model = LightFM()
#Convert data to csr matrix
data1 = csr_matrix(df2)
data1.toarray()
#Fit model with data
model.fit(data1)

In [61]:
#List all phones for future reference
phones = np.array(["Huawei", "Xiaomi", "Vivo", "Apple", "Samsung", "Sony"])
#Define a function to give recommendation phones to our users
def sample_recommendation(model, data, user_ids):

    #Get number of users and phones in our data
    n_users, n_items = data1.shape

    #Generate recommendations for each user we input
    for user_id in user_ids:

        #Get phones that our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))

        #Rank them
        top_phones = phones[np.argsort(-scores)]

        print(top_phones)

#Print out result for all our users
print('sample phone recommendations for our users are')
print('Olivia')
sample_recommendation(model, data1, [0])
print('ZJY')
sample_recommendation(model, data1, [1])
print('LHR')
sample_recommendation(model, data1, [2])
print('ZHC')
sample_recommendation(model, data1, [3])
print('ZYC')
sample_recommendation(model, data1, [4])
print('Tianen')
sample_recommendation(model, data1, [5])

sample phone recommendations for our users are
Olivia
['Apple' 'Xiaomi' 'Samsung' 'Huawei' 'Sony' 'Vivo']
ZJY
['Apple' 'Xiaomi' 'Samsung' 'Huawei' 'Vivo' 'Sony']
LHR
['Apple' 'Xiaomi' 'Huawei' 'Samsung' 'Vivo' 'Sony']
ZHC
['Apple' 'Xiaomi' 'Samsung' 'Huawei' 'Vivo' 'Sony']
ZYC
['Xiaomi' 'Apple' 'Samsung' 'Huawei' 'Vivo' 'Sony']
Tianen
['Xiaomi' 'Apple' 'Samsung' 'Huawei' 'Sony' 'Vivo']


#My References:
1. https://towardsdatascience.com/item-based-collaborative-filtering-in-python-91f747200fab
2. https://blog.csdn.net/qq_42306860/article/details/97140978
3. https://stackoverflow.com/questions/40323321/creating-a-sparse-matrix-with-lightfm-and-print-predictions